In [ ]:
"""
experiments.ipynb

File for running lottery ticket experiments.

Authors: Jordan Bourdeau, Casey Forey
Date Created: 3/8/24
"""
%load_ext tensorboard
import copy
import functools
from importlib import reload
import tensorflow as tf

from src.harness import dataset as ds
from src.harness import experiment
from src.harness import history
from src.harness import paths
from src.harness import pruning
from src.harness import rewind
from src.harness import training as train
from tensorflow.python.client import device_lib 

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(device_lib.list_local_devices())

## Run Experiments

In [ ]:
reload(ds)
reload(experiment)
reload(paths)
reload(pruning)
reload(rewind)
reload(train)

experiment_directory: str = 'testing_experiment'
experiment_summary: history.ExperimentSummary = experiment.run_experiments(
    starting_seed=0,
    num_experiments=1, 
    experiment_directory=experiment_directory,
    get_experiment_parameters=functools.partial(experiment.get_lenet_300_100_experiment_parameters, ds.Datasets.MNIST, 0.2, 0.65),
    experiment=experiment.run_iterative_pruning_experiment,
)

In [ ]:
for experiment in experiment_summary.experiments.values():
    for round in experiment.pruning_rounds:
        print(round)